### Multi-Model Performance Evaluation ###

In [2]:
import os
import numpy as np
import pandas as pd
from typing import List
import glob

# Binary perfomance metrics
from sklearn.metrics import confusion_matrix

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.performance import Performance, binary_performance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_root = os.environ.get('DATA')
data_dir = os.path.join(data_root, 'hcp')
model_dir = os.path.join(data_dir, 'models')
output_file_list = glob.glob(os.path.join(model_dir, 'hcp_predictions_*.parquet'))
print(*output_file_list, sep='\n')
pred_col_list = ['pred_mh', 'pred_ip', 'pred_op']

/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_13b.parquet
/app/data/hcp/models/hcp_predictions_20250520_gpt-4o.parquet
/app/data/hcp/models/hcp_predictions_ollama_20250520_llama2_7b.parquet


In [25]:
df = output_df.copy()
train_df = df.loc[df['dset'] == 'train'].\
        reset_index(drop=True).\
        astype({'mental_health': int, 'inpatient': int, 'outpatient': int})
# Combine mh+ip
train_df = train_df.assign(mhpip=train_df['mental_health'] + train_df['inpatient'])
                    assign(mh_ip=train_df['mhpip'].apply(lambda p: 1 if p==2 else 0))


KeyError: 'mhpip'

In [24]:
display(train_df.head())

,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,model,mhpip
0,97840-81,Vheda Health,Developer of an integrated healthcare platform...,0,0,0,train,0,1,1,1,llama2_7b,0
1,97356-34,Referral Solutions Group,Developer of an online platform designed to he...,0,0,0,train,0,1,1,1,llama2_7b,0
2,96958-00,Franklin Community Health Network,Provider of evergreen behavioral and emergency...,1,1,1,train,0,1,1,1,llama2_7b,2
3,87934-24,Forefront TeleCare,Provider of virtual behavioral health care int...,1,0,1,train,0,1,1,1,llama2_7b,1
4,83377-27,Cone Health,Provider of not-for-profit healthcare network ...,1,1,2,train,0,1,1,1,llama2_7b,2


In [20]:
display(df)

,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,model
0,97840-81,Vheda Health,Developer of an integrated healthcare platform...,0,0,0,train,0,1,1,1,llama2_7b
1,97356-34,Referral Solutions Group,Developer of an online platform designed to he...,0,0,0,train,0,1,1,1,llama2_7b
2,96958-00,Franklin Community Health Network,Provider of evergreen behavioral and emergency...,1,1,1,train,0,1,1,1,llama2_7b
3,87934-24,Forefront TeleCare,Provider of virtual behavioral health care int...,1,0,1,train,0,1,1,1,llama2_7b
4,83377-27,Cone Health,Provider of not-for-profit healthcare network ...,1,1,2,train,0,1,1,1,llama2_7b
...,...,...,...,...,...,...,...,...,...,...,...,...
182,107240-50,Alvarado Parkway Institute,Operator of a psychiatric health care facility...,1,1,1,train,0,1,1,1,llama2_7b
183,10468-27,Behavioral Centers of America,Provider of psychiatric healthcare services. T...,1,1,1,train,0,1,1,1,llama2_7b
184,10195-21,IASIS Healthcare,Owner and operator of medium-sized acute care ...,1,1,1,train,0,1,1,1,llama2_7b
185,10127-62,Quest Diagnostics (NYS: DGX),Quest Diagnostics is a leading independent pro...,1,0,0,train,0,1,1,1,llama2_7b


In [22]:
# Comparisons for test set with ground truth
performance_df_list = []
for output_file in output_file_list:
    output_df = pd.read_parquet(output_file)

In [14]:
display(train_df.head())

,id,name,description,mental_health,inpatient,outpatient,dset,pred_mh,pred_ip,pred_op,verified_op,model
0,97840-81,Vheda Health,Developer of an integrated healthcare platform...,0,0,0,train,0,1,1,1,llama2_7b
1,97356-34,Referral Solutions Group,Developer of an online platform designed to he...,0,0,0,train,0,1,1,1,llama2_7b
2,96958-00,Franklin Community Health Network,Provider of evergreen behavioral and emergency...,1,1,1,train,0,1,1,1,llama2_7b
3,87934-24,Forefront TeleCare,Provider of virtual behavioral health care int...,1,0,1,train,0,1,1,1,llama2_7b
4,83377-27,Cone Health,Provider of not-for-profit healthcare network ...,1,1,2,train,0,1,1,1,llama2_7b


In [ ]:
display(Performance(data=df.copy()).binary_performance(true_col='mental_health', 
                                                       pred_col='pred_mh'))